<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/codes/blob/main/bitsnbytes_zero_shotprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### langchain installation

In [ ]:
# !pip install langchain
# !pip install langchain_community
# !pip install bitsandbytes
# !pip install git+https://github.com/huggingface/accelerate.git
# !pip install git+https://github.com/huggingface/transformers.git


# # !pip uninstall transformers
# # !pip uninstall bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-19p2t2ji
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-19p2t2ji
  Resolved https://github.com/huggingface/accelerate.git to commit 8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.1.0.

### importing libraries

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from huggingface_hub import HfFolder
from google.colab import userdata
import torch

### intializing variables

In [ ]:
HfFolder.save_token(userdata.get('hf_token'))


# using quantization
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

_model_id='meta-llama/Llama-3.2-1B'

_tokenizer=AutoTokenizer.from_pretrained(_model_id)
_model=AutoModelForCausalLM.from_pretrained(_model_id,
                                            device_map='auto',
                                            quantization_config=nf4_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

### functions

In [ ]:
_template=PromptTemplate(
    input_variables=['topic'],
    template='write a short essay about {topic}'
)
pipe=pipeline('text-generation',
                   model=_model,
                   tokenizer=_tokenizer,
                   max_new_tokens=512,
              device_map='auto'
                   )

Device set to use cuda:0


In [ ]:
_llm =HuggingFacePipeline(pipeline=pipe)

chain=LLMChain(llm=_llm, prompt=_template)

result=chain.run(topic='climate change')

<ipython-input-6-d576b005530f>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  _llm =HuggingFacePipeline(pipeline=pipe)
<ipython-input-6-d576b005530f>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=_llm, prompt=_template)
<ipython-input-6-d576b005530f>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain.run(topic='climate change')
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

write a short essay about climate change
If you are looking for a paper on climate change, you have come to the right place. We have prepared a paper that will help you write a short essay on climate change. This paper is based on the research of climate change. This research paper is written by a team of experts. We have also provided you with a sample of a short essay on climate change.
Climate change is a very serious issue. Climate change is caused by the burning of fossil fuels. This is a serious issue because the burning of fossil fuels has been increasing over the past 50 years. This is because the burning of fossil fuels is causing global warming. This is a serious issue because global warming is causing a rise in the temperature of the earth. This is a serious issue because global warming is causing a rise in the number of hurricanes in the Atlantic Ocean.
Climate change is also a serious issue because it is causing the oceans to rise. This is because the oceans are rising bec

In [ ]:
result.strip()

'write a short essay about climate change\nIf you are looking for a paper on climate change, you have come to the right place. We have prepared a paper that will help you write a short essay on climate change. This paper is based on the research of climate change. This research paper is written by a team of experts. We have also provided you with a sample of a short essay on climate change.\nClimate change is a very serious issue. Climate change is caused by the burning of fossil fuels. This is a serious issue because the burning of fossil fuels has been increasing over the past 50 years. This is because the burning of fossil fuels is causing global warming. This is a serious issue because global warming is causing a rise in the temperature of the earth. This is a serious issue because global warming is causing a rise in the number of hurricanes in the Atlantic Ocean.\nClimate change is also a serious issue because it is causing the oceans to rise. This is because the oceans are rising